### **Paso 1.6 - Creación de Pipeline "pl_Copy_FULL_IPL_DATA"**

1.  Una forma mucho más eficiente de trabajar con múltiples archivos es utilizando **PARÁMETROS**. Para ello trabajaremos solo con 2 datasets, uno de origen y de destino y crearemos dos parámetros, ambos a nivel de dataset, uno en cada dataset. Reutilizaremos los datasets **ds_Source_IPL_2008_csv** y **ds_Destination_IPL_2008_json** los cuales haran referencia a un parámetro y no a un archivo. Modificaremos sus nombres:

    *   **ds_Source_IPL_csv**
    *   **ds_Destination_IPL_json**

    Crearemos el pipeline **pl_Copy_FULL_IPL_DATA** el cual utilizará las actividades **Lookup** y **For Each**  

#### Agregar parámetros a datasets, crear un nuevo Pipeline y agregar la actividad "Lookup"

Añadiremos un parámetro para el archivo **ds_Source_IPL_2008_json** que es el archivo que utilizaremos como dataset de destino para todos los archivos. Más adelante modificaremos su nombre

<center><img src="https://i.postimg.cc/KvJP6N2V/adf280.png"></center> 
<center><img src="https://i.postimg.cc/sx242VtY/adf281.png"></center>

Añadiremos un parámetro para el archivo **ds_Source_IPL_2008_csv** que es el archivo que utilizaremos como dataset de origen para todos los archivos. Más adelante modificaremos su nombre

<center><img src="https://i.postimg.cc/DzKQXQ9y/adf282.png"></center>
<center><img src="https://i.postimg.cc/SK76724L/adf283.png"></center>

Modificamos los nombres de nuestros dataset de origen y destino. Quedarán de la siguiente manera:
    
    *   ds_Source_IPL_csv
    *   ds_Destination_IPL_json

<center><img src="https://i.postimg.cc/5yHQGjhR/adf284.png"></center>

Crearemos un nuevo pipeline, con nuevos ajustes que nos permita mayor eficiencia al momento de lanzar nuestro proceso. Nuestro pipeline llevará el nombre de **pl_Copy_Full_IPL_DATA** y al cual le agregamos una actividad **Lookup**

<center><img src="https://i.postimg.cc/Hk18ZYB6/adf285.png"></center>

Eliminaremos los archivos que se almacenaron en el contenedor **raw** de nuestro ADLS y cargaremos el archivo **Files.txt**, que es basicamente un archivo que almacena los nombres de todos los archivos que queremos extraer desde el sitio de Github

<center><img src="https://i.postimg.cc/jjd1fb0j/adf286.png"></center>
<center><img src="https://i.postimg.cc/KzCpC3KK/adf287.png"></center>

Crearemos un nuevo dataset que haga referencia hacia el archivo **Files.txt** que recien subimos a nuestro  ADLS, al contenedor **raw**. Este dataset llevará el nombre de **ds_listOfFiles**

<center><img src="https://i.postimg.cc/TYVNHwTw/adf288.png"></center>
<center><img src="https://i.postimg.cc/hPc1v0qD/adf289.png"></center>
<center><img src="https://i.postimg.cc/MphcMWj7/adf290.png"></center>

Realizamos ajustes a nuestra actividad **Lookup** de nuestro pipeline **pl_Copy_Full_IPL_DATA**

<center><img src="https://i.postimg.cc/XvSJNQTx/adf291.png"></center>
<center><img src="https://i.postimg.cc/VkMfhXdt/adf292.png"></center>

Vamos a **Validar** y luego ejecutar el pipeline utilizando **Debug**. Revisaremos que nos devuelve el **Output**

<center><img src="https://i.postimg.cc/hPRcdfCP/adf293.png"></center>
<center><img src="https://i.postimg.cc/bw3hWLqs/adf294.png"></center>


#### Agregar la actividad "ForEach"

Agregaremos una actividad **ForEach** a nuestro Pipeline y le realizaremos los ajustes necesarios

<center><img src="https://i.postimg.cc/1t4pyBgW/adf295.png"></center>
<center><img src="https://i.postimg.cc/Qt0KKT7F/adf296.png"></center>

Utilizaremos el contenido dinámico **@activity('Lookup All File Names').output.value** en la configuración
de la actividad ForEach, esto quiere decir, que de la actividad previa **Lookup All File Name** extraera
solo **value**. Corresponde al array que contiene los nombres de los archivos que utilizaremos.

Esto es el **Output** de la actividad previa **Lookup**
```
{
    "count":15,
    "value": [
        {
            "Files":"ipl 2008.csv"
        }
        {
            "Files":"ipl 2009.csv"            
        }
        ...
        ...
    ]

}
```
<center><img src="https://i.postimg.cc/dtzkCvhj/adf297.png"></center>
<center><img src="https://i.postimg.cc/8CG7yHBF/adf298.png"></center>

Ahora, editaremos la tarea que realizará nuestro **ForEach**. Agregamos la actividad **Copy Data**.

<center><img src="https://i.postimg.cc/CKymKWCR/adf299.png"></center>

Para la configuración del origen **Source** se creará el contenido dinámico **@items().Files** que captura los valores de **Files**. Esto es el **Output** de la actividad previa **Lookup**
```
{
    "count":15,
    "value": [
        {
            "Files":"ipl 2008.csv"
        }
        {
            "Files":"ipl 2009.csv"            
        }
        ...
        ...
    ]

}
```
Este objeto **json** lo asignamos como contenido dinámico en la configuración de la actividad **For each**, en **Settings** en la opción de **Items**. Es por eso el motivo que utilizamos **@items**, estamos capturando los valores que asignamos a esa opción.


Por cada valor de **File** que se tenga, se irá heredando al parámetro del dataset de origen **dataset().SourceFileName**,
por tanto, se irá heredando cada valor de **File** como nombres de archivos para el dataset de origen

<center><img src="https://i.postimg.cc/NfSxNWzd/adf300.png"></center>

Para la configuración del destino **Sink** se creará el contenido dinámico **@replace(@items().Files,'.csv','.json')**
que tendrá la misma función que el contenido dinámico para el Origen, solo que en este caso se reemplazará su extensión
por una JSON.

<center><img src="https://i.postimg.cc/QdkwRQW9/adf301.png"></center>
<center><img src="https://i.postimg.cc/FFM6zWyk/adf302.png"></center>

Vamos a **Validar** y luego ejecutar el pipeline utilizando **Debug**.

<center><img src="https://i.postimg.cc/4x7nP3KS/adf303.png"></center>

Si revisamos nuestro ADLS, el contenedor **raw** se veria de la siguiente manera

<center><img src="https://i.postimg.cc/nL9RnqXG/adf304.png"></center>

